# 3-2 Mid-level API: Demonstration

The examples below use mid-level APIs in TensorFlow to implement a linear regression model.

Mid-level API includes model layers, loss functions, optimizers, data pipelines, feature columns, etc.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers,losses,metrics,optimizers


# Print time stamps
@tf.function
def printbar():
    ts = tf.timestamp()
    today_ts = ts%(24*60*60)

    hour = tf.cast(today_ts//3600+8,tf.int32)%tf.constant(24)
    minite = tf.cast((today_ts%3600)//60,tf.int32)
    second = tf.cast(tf.floor(today_ts%60),tf.int32)
    
    def timeformat(m):
        if tf.strings.length(tf.strings.format("{}",m))==1:
            return(tf.strings.format("0{}",m))
        else:
            return(tf.strings.format("{}",m))
    
    timestring = tf.strings.join([timeformat(hour),timeformat(minite),
                timeformat(second)],separator = ":")
    tf.print("=========="*8,end = "")
    tf.print(timestring)

In [2]:
# Number of samples
n = 800

# Generate testing dataset
X = tf.random.uniform([n,2],minval=-10,maxval=10) 
w0 = tf.constant([[2.0],[-1.0]])
b0 = tf.constant(3.0)
Y = X@w0 + b0 + tf.random.normal([n,1],mean = 0.0,stddev= 2.0)  # @ represents matrix multiplication; add Gaussian noise

# Creating pipeline of input data
ds = tf.data.Dataset.from_tensor_slices((X,Y)) \
     .shuffle(buffer_size = 1000).batch(100) \
     .prefetch(tf.data.experimental.AUTOTUNE)  

# Defining optimizer
optimizer = optimizers.SGD(learning_rate=0.001)

In [3]:
linear = layers.Dense(units = 1)
linear.build(input_shape = (2,)) 

@tf.function
def train(epoches):
    for epoch in tf.range(1,epoches+1):
        L = tf.constant(0.0) # L is used for recording loss values
        for X_batch,Y_batch in ds:
            with tf.GradientTape() as tape:
                Y_hat = linear(X_batch)
                loss = losses.mean_squared_error(tf.reshape(Y_hat,[-1]),tf.reshape(Y_batch,[-1]))
            grads = tape.gradient(loss,linear.variables)
            optimizer.apply_gradients(zip(grads,linear.variables))
            L = loss
        
        if(epoch%100==0):
            printbar()
            tf.print("epoch =",epoch,"loss =",L)
            tf.print("w =",linear.kernel)
            tf.print("b =",linear.bias)
            tf.print("")

train(500)

================================================================================16:28:49
epoch = 100 loss = 6.13818932
w = [[2.00223255]
 [-1.01345801]]
b = [2.41117597]

================================================================================16:28:52
epoch = 200 loss = 5.00730753
w = [[2.00545073]
 [-1.0149585]]
b = [2.9106]

================================================================================16:28:54
epoch = 300 loss = 5.12224627
w = [[2.00464392]
 [-1.01466823]]
b = [3.0112443]

================================================================================16:28:57
epoch = 400 loss = 3.67764401
w = [[2.00470614]
 [-1.01397789]]
b = [3.03155828]

================================================================================16:28:59
epoch = 500 loss = 5.27416801
w = [[2.00681973]
 [-1.0149914]]
b = [3.03578353]

